In [78]:
import polars as pl
import json

# Create table of places

## Load data for places

In [79]:
places = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/places.csv"
)
places_to_types = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/places_to_types.csv"
)
feature_types = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/feature_types.csv"
)
event_types = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/events_to_types.csv"
)
locs = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/locations.csv"
)
places_to_attestations = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/places_to_attestation.csv"
)
places_to_establishment = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/places_to_establishment.csv",
    ignore_errors=True,
)
feature_class = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/feature_class.csv"
)
# Remove duplicate place entries
locs = locs.unique(subset=["place_id"], keep="first")

## Join tables to create places table

In [80]:
from re import sub


pt_df = places.join(places_to_types, on="place_id", how="left")
pt_df = pt_df.join(locs, on="place_id", how="left").unique(
    subset=["place_id"], keep="first"
)
pt_df = pt_df.rename({"ft_id": "yft_id"})
pt_df = pt_df.join(feature_types, on="yft_id", how="left", suffix="_ft_tp")
pt_df = pt_df.join(places_to_attestations, on="place_id", how="left")
pt_df = pt_df.join(places_to_establishment, on="place_id", how="left")
pt_df = pt_df.join(feature_class, on="fct_id", how="left")
pt_df = pt_df.select(
    [
        "place_id",
        "tr_title",
        "ch_pinyin",
        "latitude",
        "longitude",
        "attestation",
        "ch_title",
        "feature_class",
        "en_title",
        "est_year",
    ]
)
pt_df = pt_df.unique()
pt_df.head()

place_id,tr_title,ch_pinyin,latitude,longitude,attestation,ch_title,feature_class,en_title,est_year
str,str,str,f64,f64,str,str,str,str,i64
"""yrdb4058""","""綏平堡""","""suiping bao""",36.234135,102.007131,"""Fortifications_Year612_Merge_4…",null,null,null,612
"""yrdb2306""","""破虎堡""","""pohu bao""",112.519212,40.226051,"""us_80634""","""堡""","""habitations""","""fortress""",1820
"""yrdb3513""","""黑城工""","""heichenggong""",101.87668,36.088554,"""us_80330""",null,null,null,1820
"""yrdb1097""","""張亮堡""","""zhangliang bao""",106.363304,38.476113,"""us_70230""","""堡""","""habitations""","""fortress""",1582
"""yrdb4472""","""靈平寨""","""lingping zhai""",36.288763,106.131745,"""Fortifications_Year741_105""",null,null,null,741


In [81]:
print(places.shape)
print(pt_df.shape)
pt_df = pt_df.sort("est_year", nulls_last=True).unique(
    subset=["place_id"], keep="first"
)
print(pt_df.shape)

(4225, 3)
(4738, 10)
(4225, 10)


## Split into Upstream and Downstream data

In [82]:
upstream = pt_df.filter(pl.col("attestation").str.starts_with("us"))
downstream = pt_df.filter(pl.col("attestation").str.starts_with("ds"))

In [83]:
import requests

# Join Upstream places with infromation
eras = requests.get(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/eras.geojson"
).json()["features"]
dyns = requests.get(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/dynasties.geojson"
).json()["features"]
dyns_df = pl.DataFrame([f["properties"] for f in dyns], orient="row")
eras_df = pl.DataFrame([f["properties"] for f in eras], orient="row")
dyns_df.head()

id,name_en,name_ch,start_cert,start_date,end_cert,end_date
i64,str,str,str,i64,str,i64
1,"""Xia dynasty ""","""夏朝""","""n""",-2070,"""n""",-1600
2,"""Shang dynasty ""","""商朝 ""","""n""",-1600,"""n""",-1046
3,"""Zhou dynasty ""","""周朝 ""","""n""",-1046,"""y""",-256
4,"""Qin dynasty ""","""秦朝 ""","""y""",-221,"""y""",-207
5,"""Han dynasty ""","""漢朝 ""","""y""",-202,"""y""",220


In [84]:
eras_df.head()

id,monarch,era_name_en,era_name_ch,era_start,era_end
str,str,str,str,i64,i64
"""I0019""","""I0001""","""Jianlong""","""建隆""",960,963
"""I0020""","""I0001""","""Qiande""","""乾德""",963,968
"""I0021""","""I0001""","""Kaibao""","""開寶""",968,976
"""I0022""","""I0002""","""Taipingxingguo""","""太平興國""",976,984
"""I0023""","""I0002""","""Yongxi""","""雍熙""",984,987


In [85]:
dyns_df = dyns_df.select(["name_en", "name_ch", "start_date", "end_date"]).rename(
    {"name_en": "dynasty_en", "name_ch": "dynasty_ch"}
)

In [88]:
dyn_en = []
dyn_ch = []
for row in upstream.iter_rows():
    est_year = row[9]
    if est_year is None:
        dyn_en.append(None)
        dyn_ch.append(None)
        continue
    matched_dyn = dyns_df.filter(
        (pl.col("start_date") <= est_year) & (pl.col("end_date") >= est_year)
    )
    if matched_dyn.is_empty():
        dyn_en.append(None)
        dyn_ch.append(None)
    else:
        dyn_en.append(matched_dyn[0, "dynasty_en"].strip())
        dyn_ch.append(matched_dyn[0, "dynasty_ch"].strip())

upstream = upstream.with_columns(
    [pl.Series("dynasty_en", dyn_en), pl.Series("dynasty_ch", dyn_ch)]
)

upstream.head()

place_id,tr_title,ch_pinyin,latitude,longitude,attestation,ch_title,feature_class,en_title,est_year,dynasty_en,dynasty_ch
str,str,str,f64,f64,str,str,str,str,i64,str,str
"""yrdb3796""","""三不通""","""sanbutong""",107.716062,35.549001,"""us_81156""",null,null,null,1820,"""Qing dynasty""","""清朝"""
"""yrdb851""","""長武縣""","""changwu xian""",107.749318,35.110861,"""us_70274""","""縣""","""administrative units""","""county""",1330,"""Yuan dynasty""","""元朝"""
"""yrdb396""","""口前村""","""kouqian cun""",112.621455,39.667123,"""us_80845""","""村""","""habitations""","""village""",1820,"""Qing dynasty""","""清朝"""
"""yrdb1480""","""枳道""","""zhi dao""",109.03359,34.250559,"""us_20094""","""道""","""administrative units""","""circuit""",281,"""Jin dynasty""","""晉朝"""
"""yrdb2461""","""羊腸倉""","""yangchangcang""",111.890922,38.305131,"""us_20194""",null,null,null,null,null,null


In [89]:
# convert upstream to json and write to upstream-data.json

upstream_data = []

for row in upstream.iter_rows():
    id = row[0]
    hz = row[1]
    py = row[2]
    x_coor = row[4]
    y_coor = row[3]
    date = row[9]
    regime = row[10]
    regime_ch = row[11]
    name_type = row[6]
    name_type_en = row[8]
    name_class_en = row[7]

    place_json = {
        "id": id,
        "hz": hz,
        "py": py,
        "x_coor": x_coor,
        "y_coor": y_coor,
        "date": date,
        "regime": regime,
        "regime_ch": regime_ch,
        "name_type": name_type,
        "name_type_en": name_type_en,
        "name_class_en": name_class_en,
    }
    upstream_data.append(place_json)

with open("upstream-data.json", "w", encoding="utf-8") as f:
    json.dump(upstream_data, f, ensure_ascii=False, indent=4)

# Create table for events

## Load data for events

In [ ]:
import numpy as np

events = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/events.csv"
)
event_types = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/event_types.csv"
)
event_cats = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/event_categories.csv"
)

events_to_places = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/events_to_places.csv"
).rename({"event": "event_id"})
events_to_types = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/events_to_types.csv"
)
events_to_sources = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/sources_to_events.csv",
    ignore_errors=True,
)
sources = pl.read_csv(
    "https://raw.githubusercontent.com/YellowRiverDatabase/geodata/refs/heads/main/relational-data/sources.csv",
    null_values=["", "nan", "NaN", "NA", "null"],
    ignore_errors=True,
)
events_to_sources.head()

src_to_evt_id,source_id,event_id
str,i64,str
"""srev_1""",11001001,"""ev_1"""
"""srev_2""",11001002,"""ev_2"""
"""srev_3""",11001003,"""ev_3"""
"""srev_4""",11001004,"""ev_4"""
"""srev_5""",11001005,"""ev_5"""


In [78]:
events.head(1)

event_id,ch_date,western_date,description,notes
str,str,f64,str,str
"""ev_1""","""史前时代""",-2356.0,null,null


In [79]:
event_types.head(1)

event_type_id,zh_ch_title,en_title,en_type,evc_id,description
str,str,str,str,str,str
"""evtype_1""","""溢""","""yi""","""Flood""","""evc_1""","""Any time there is a zhang 漲(ra…"


In [80]:
events_to_places.head(1)

evtp_id,place_id,event_id,attestation
str,str,str,str
"""evtp_1""","""yrdb2""","""ev_858""","""ds_1019"""


In [81]:
events_to_types.head(1)

evetotyp_id,event_id,event_type_id
str,str,str
"""evetotyp_1""","""ev_1""","""evtype_1"""


In [82]:
event_cats.head(1)

evc_id,zh_cn_category,en_category
str,str,str
"""evc_1""","""水災""","""Disasters"""


In [83]:
sources.head(1)

source_id,source,page,chinese_date,western_date,old_placename_chinese,modern_placename_chinese,event_type_chinese,event_name,event_description,primary_source_1,primary_source_2,notes
i64,str,str,str,str,str,str,str,str,str,str,str,str
10100001,"""HDSJ""",null,"""传说时代""","""约-21世纪初""",null,null,null,"""大禹治水""","""传说中的尧舜时代，黄河流域发生大洪水，为制止洪水泛滥，尧召集…",null,null,null


In [ ]:
print(f"events length: {events.shape}")
ev_df = events.join(events_to_types, on="event_id", how="left", suffix="_ett")
ev_df = ev_df.unique(keep="first")
print(f"ev_df length: {ev_df.shape}")
ev_df = ev_df.join(events_to_places, on="event_id", how="left", suffix="_etp")

ev_df = ev_df.unique(keep="first")
ev_df = ev_df.join(events_to_sources, on="event_id", how="left")
ev_df = ev_df.join(sources, on="source_id", how="left")
ev_df = ev_df.join(event_types, on="event_type_id", how="left")
ev_df = ev_df.unique(subset=["event_id", "western_date", "place_id", "en_type"])
ev_df = ev_df.join(event_cats, on="evc_id", how="left", suffix="_cat")
print(f"ev_df len after join2: {ev_df.shape}")

events length: (3754, 5)
ev_df length: (5349, 7)
ev_df len after join2: (15639, 31)


In [91]:
ev_df.sample(2)

event_id,ch_date,western_date,description,notes,evetotyp_id,event_type_id,evtp_id,place_id,attestation,src_to_evt_id,source_id,source,page,chinese_date,western_date_right,old_placename_chinese,modern_placename_chinese,event_type_chinese,event_name,event_description,primary_source_1,primary_source_2,notes_right,zh_ch_title,en_title,en_type,evc_id,description_right,zh_cn_category,en_category
str,str,f64,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ev_2381""","""清圣祖康熙十五年""",1676.0,null,null,"""evetotyp_3331""","""evtype_2""","""evtp_550""","""yrdb139""","""ds_608""","""srev_3672""",10410137,"""HHNB""",null,"""清圣祖康熙十五年""","""1676""",null,null,"""修""",null,"""筑兰阳铜瓦厢月堤。（河南通志）""","""决于二十年。""",null,null,"""決""","""jue""","""Breach""","""evc_1""","""Any time there is a beng 崩(sud…","""水災""","""Disasters"""
"""ev_2264""","""清世祖顺治十四年""",1657.0,null,null,"""evetotyp_3164""","""evtype_16""","""evtp_213""","""yrdb41""","""ds_1267""","""srev_3511""",10410036,"""HHNB""",null,"""清世祖顺治十四年""","""1657""",null,null,"""修""",null,"""浚仪封县三家庄河长一千丈，以杀北来水势，而黄渡河涸为田。（目…",null,null,null,"""治""","""zhi""","""Management""","""evc_2""","""Very huge and general. 1) Any…","""水利""","""Management"""


In [93]:
yrdb_events = []

for entry in pt_df.iter_rows():
    yrdb_id = entry[0]
    tr_title = entry[1]
    ch_pinyin = entry[2]
    lat = entry[7]
    long = entry[8]
    events = []
    for event in ev_df.filter(pl.col("place_id") == yrdb_id).iter_rows():
        event_id = event[0]
        en_date_start = event[2]
        ch_date = event[1]
        en_cat = event[30]
        en_type = event[26]
        en_title = event[25]
        ch_cat = event[29]
        ch_title = event[24]
        description = event[20]

        event_dict = {
            "event_id": event_id,
            "en_date_start": en_date_start,
            "ch_date": ch_date,
            "en_cat": en_cat,
            "en_type": en_type,
            "en_title": en_title,
            "ch_cat": ch_cat,
            "ch_title": ch_title,
            "description": description,
        }
        events.append(event_dict)

    # Build place dictionary with nested events
    place_dict = {
        "yrdb_id": yrdb_id,
        "tr_title": tr_title,
        "ch_pinyin": ch_pinyin,
        "lat": lat,
        "long": long,
        "events": events,
    }
    yrdb_events.append(place_dict)

# Write to JSON file
with open("yrdb_events.json", "w", encoding="utf-8") as f:
    json.dump(yrdb_events, f, ensure_ascii=False, indent=2)